In [1]:
import os

In [2]:
%pwd

'd:\\Kidney\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Kidney'

In [5]:
#  Update the entity
#'''Entity nothing but the retun type of anu funtion Below is the return type'''

In [6]:
#Update the entity after this we have to copy and past in the src file as per the readme workflow
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

    unzip_dir: Path

In [7]:
# Update the configuration manager in src config
#Here we are importing the two path 
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
# Update the configuration manager in src config
#we create the class and inside the class we create the constructor
class ConfigurationManager:
    # This  confi and params file path i am initlizing inside the class variable
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
#using this read_yaml funtion i reading this two file config and params yaml & storing inside the config and params

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
#Now i have to prepare data ingestion related configuration


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
#Now i am updating the components 
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
 #fist we creat the class data ingestion 
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
 # Now i will create the first funtion download data       
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            #This my source URL 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
#Now i need to Update the pipeline the pipeline means the step we are runing the funtion step1:call download_file step2:call extract_zip_file
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-28 17:48:24,289: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-28 17:48:24,289: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-28 17:48:24,295: INFO: common: created directory at: artifacts]
[2023-11-28 17:48:24,296: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-28 17:48:24,297: INFO: 2204213689: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=9e0d4a5e-98eb-4a16-902c-a675b0ff9cd3
To: d:\Kidney\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:10<00:00, 5.44MB/s]

[2023-11-28 17:48:36,538: INFO: 2204213689: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
